In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

In [2]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [7]:
TRAINING_DIR = "./train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "./test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 1794 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [4]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [5]:
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [8]:
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=validation_generator,
                    callbacks=[checkpoint])

Epoch 1/10
180/180 [==============================] - 77s 425ms/step - loss: 0.1454 - acc: 0.9398 - val_loss: 0.1836 - val_acc: 0.9536
Epoch 2/10
180/180 [==============================] - 83s 460ms/step - loss: 0.1425 - acc: 0.9404 - val_loss: 0.1776 - val_acc: 0.9433
Epoch 3/10
180/180 [==============================] - ETA: 0s - loss: 0.1374 - acc: 0.9470INFO:tensorflow:Assets written to: model2-003.model\assets


INFO:tensorflow:Assets written to: model2-003.model\assets


180/180 [==============================] - 78s 432ms/step - loss: 0.1374 - acc: 0.9470 - val_loss: 0.1385 - val_acc: 0.9433
Epoch 4/10
180/180 [==============================] - 77s 426ms/step - loss: 0.1499 - acc: 0.9398 - val_loss: 0.1913 - val_acc: 0.9433
Epoch 5/10
180/180 [==============================] - 80s 441ms/step - loss: 0.1441 - acc: 0.9420 - val_loss: 0.4168 - val_acc: 0.7938
Epoch 6/10
180/180 [==============================] - ETA: 0s - loss: 0.1528 - acc: 0.9404INFO:tensorflow:Assets written to: model2-006.model\assets


INFO:tensorflow:Assets written to: model2-006.model\assets


180/180 [==============================] - 83s 461ms/step - loss: 0.1528 - acc: 0.9404 - val_loss: 0.1135 - val_acc: 0.9588
Epoch 7/10
180/180 [==============================] - 77s 429ms/step - loss: 0.1291 - acc: 0.9493 - val_loss: 0.2463 - val_acc: 0.9072
Epoch 8/10
180/180 [==============================] - 73s 405ms/step - loss: 0.1348 - acc: 0.9482 - val_loss: 0.2085 - val_acc: 0.9381
Epoch 9/10
180/180 [==============================] - 72s 400ms/step - loss: 0.1339 - acc: 0.9493 - val_loss: 0.4258 - val_acc: 0.8196
Epoch 10/10
180/180 [==============================] - 75s 415ms/step - loss: 0.1367 - acc: 0.9443 - val_loss: 0.2550 - val_acc: 0.9175


In [9]:
import cv2
import numpy as np
from keras.models import load_model

results={0:'without mask',1:'mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}

rect_size = 4
cap = cv2.VideoCapture(0) 


haarcascade = cv2.CascadeClassifier('/home/user_name/.local/lib/python3.6/site-packages/cv2/data/haarcascade_frontalface_default.xml')

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = face_cascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break

cap.release()

cv2.destroyAllWindows()

1/1 [==============================] - 0s 28ms/step


In [10]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
#from tensorflow.contrib import lite

In [11]:
keras_file="model.h5"
keras.models.save_model(model,keras_file)

In [12]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("model.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\ADMIN\AppData\Local\Temp\tmprdvebf46\assets


INFO:tensorflow:Assets written to: C:\Users\ADMIN\AppData\Local\Temp\tmprdvebf46\assets


26295544